In [ ]:
import json

from smart_open import open
import pandas as pd
import os 
from gretel_client import configure_session

from gretel_client import submit_docker_local
from gretel_client.projects import create_or_get_unique_project

configure_session(api_key="grtub9ad1872f577fb7f6392e130b7725b4abe73b232d07b54d35296de5513dd60b3", cache="no", validate=True)


In [ ]:
def insert_rows(folder_path, value_one, value_two):
    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Insert two rows with constant values
            new_rows = pd.DataFrame({"one": [value_one, value_one], "two": [value_two, value_two]})
            df = pd.concat([new_rows, df], ignore_index=True)
            
            # Write back to the same CSV file
            df.to_csv(file_path, index=False)

# Example usage
folder_path = "path/to/csv/files"
value_one = "a"
value_two = "b"
insert_rows(folder_path, value_one, value_two)

In [9]:
folder_path = "C:\D\Documents\onlab\_testsignals110"
file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path) if file_name.endswith(".csv")]
data_frames = [pd.read_csv(file_path) for file_path in file_paths]
df = pd.concat(data_frames, ignore_index=True)
df.to_csv("training_data.csv", index=False)

In [13]:
# Load config and set training parameters
from gretel_client.projects.models import read_model_config

config = read_model_config("C:\D\Documents\onlab\gretel\model.yml")

config["name"] = "dgan-for-ctg"
config["models"][0]["timeseries_dgan"]["generate"] = {"num_records": 3000}
config["models"][0]["timeseries_dgan"]["data_source"] = "training_data.csv"

In [ ]:
# Create a project and train the synthetic data model

project = create_or_get_unique_project(name="synthetic-data-local")
model = project.create_model_obj(model_config=config)
run = submit_docker_local(model, output_dir="tmp/")

In [ ]:
# View the generated synthetic data
for i in range(0,100):
    synthetic_df = pd.read_csv("tmp/data_preview.gz", compression="gzip")
    synthetic_df.to_csv(f"C:\D\Documents\onlab\gretel\gretel_gen{i}.csv")

In [ ]:

# Use the trained model to create additional synthetic data
for i in range(0,100):
    record_handler = model.create_record_handler_obj(params={"num_records": 100})
    run = submit_docker_local(
        record_handler, model_path="tmp/model.tar.gz", output_dir="tmp/"
    )
    synthetic_df_new = pd.read_csv("tmp/data.gz", compression="gzip")
    synthetic_df_new.to_csv(f"C:\D\Documents\onlab\gretel\gretel_gen{i}.csv")